In [5]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
import sqlite3
import time

"""
#Create db
conn = sqlite3.connect('financial_data.db')

# Create a cursor
cursor = conn.cursor()
"""
# Create the first table
#all_time_prices_query = 
'''
CREATE TABLE all_time_prices (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

#cursor.execute(all_time_prices_query)

"""

momentum_query = '''
CREATE TABLE momentum (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

cursor.execute(momentum_query)

conn.commit()

# Close the connection
conn.close()
"""

"\n\nmomentum_query = '''\nCREATE TABLE momentum (\n    Date INTEGER PRIMARY KEY,\n    Open FLOAT,\n    High FLOAT, \n    Low FLOAT, \n    Close FLOAT, \n    Volume FLOAT,\n    Dividends FLOAT, \n    Stock Splits FLOAT, \n    Ticker TEXT NOT NULL    \n);\n'''\n\ncursor.execute(momentum_query)\n\nconn.commit()\n\n# Close the connection\nconn.close()\n"

In [7]:
#calulate the Hurst exponent of a stock
def get_hurst_exponent(time_series, max_lag):
    """Returns the Hurst Exponent of the time series"""
        
    lags = range(2, max_lag)

    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]

    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)

    return reg[0]

#Create first table with all stock info from yfinance
tick = pd.read_csv("all_stocks")
all_time_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Ticker']
momentum_cols = ['Ticker','Price',
                '1y Price Return','1y percentile', 
                '6m Price Return', '6m percentile', 
                '3m Price Return', '3m percentile',
                '1m Price Return','1m percentile',
                'Weighted Hurst Exponent']
momentum = pd.DataFrame(columns = momentum_cols)
all_time = pd.DataFrame(columns = all_time_cols)
count = 0 
for i in range(len(tick)): 
    start = time.time()    
    try:
        #Creating the df to be added to all_time_prices
        ticker = tick["0"][i]
        info = yf.Ticker(ticker).history(period='max')
        info.reset_index(inplace = True)
        info['Ticker'] = ticker

        # Append the first dataframe to the table
        all_time = pd.concat([all_time, info])

        #Creating the df to be added to momentum
        y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
        m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
        m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
        m1 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=1))

        momentum['Ticker'] = tick['0'][i]  
        try: 
            perc_change1y = ((y.iloc[-1]['Close'] - y.iloc[0]['Close'])/y.iloc[0]["Close"])
            d['1y Price Return'] = perc_change1y
            perc_change6m = ((m6.iloc[-1]['Close'] - m6.iloc[0]['Close'])/m6.iloc[0]["Close"])
            d['6m Price Return'] = perc_change6m
            perc_change3m = ((m3.iloc[-1]['Close'] - m3.iloc[0]['Close'])/m3.iloc[0]["Close"])
            d['3m Price Return'] = perc_change3m
            perc_change1m = ((m1.iloc[-1]['Close'] - m1.iloc[0]['Close'])/m1.iloc[0]["Close"])
            d['1m Price Return'] = perc_change1m

            d['Price'] = d['Close'] 
            d['1y percentile'] = 0
            d['6m percentile'] = 0
            d['3m percentile'] = 0
            d['1m percentile'] = 0
            d['Weighted Hurst Exponent'] = 0
            d = d.iloc[-1]
        except IndexError: 
            print('Index Error when producing momentum table')
            continue
        
        t = [20, 100, 300, 500, 1000]
        total_hurst = 0
        weight = 0.4
        for j in range(len(t)): 
            #Use Weighted average to determine hurst exp of the stock
            hurst_exp = get_hurst_exponent(info["Close"].values, t[j])
            #print(tick["0"][i] + " Hurst exponent with " + str(t[j]) +  ' lags: ' + str(hurst_exp))
            total_hurst += hurst_exp*weight
            if j >= 2: 
                weight = 0.1
            else: 
                weight = 0.2
            print(total_hurst)
        d['Weighted Hurst Exponent'] = total_hurst

    #except TypeError: 
    #    print("Nonetype found for: " + tick["0"][i])
    #   continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    stop = time.time()
    duration = stop-start
    print(duration)
#Get percentiles for each stock
cols = ['1y', '6m', '3m', '1m']
for c in cols: 
    for index, row in momentum.iterrows(): 
        percentile_change = stats.percentileofscore(momentum[c+ ' Price Return'], momentum[c + ' Price Return'].loc[index])
        momentum[c + ' percentile'][index] = percentile_change 
momentum['Shares To Buy'] = 0
    
#Calculate HQM score
#Get the mean of all 4 percentiles 
momentum['HQM score'] = 0
from statistics import mean
for index, row in momentum.iterrows(): 
    all_periods_p = [row['1y percentile'], row['6m percentile'], row['3m percentile'], row['1m percentile']]
    momentum['HQM score'].iloc[index] = mean(all_periods_p)


#PUSH TO DATABASE
#all_time.to_sql('all_time_prices', conn, if_exists='append', index=False)

TypeError: '<' not supported between instances of 'int' and 'Timestamp'